# PCAP L6 — Exercise 2 (Answers Notebook)

`Mission 02`: Configure probes with closures and log their data using file I/O.

**Context:** You're writing tools for an important deep-space mission and need to work with data using file I/O. This will leverage your knowledge of:
- Closures and factory functions,
- Basic text file read/write, and
- Using `bytearray` as a mutable sequence of bytes.

> 🎯 Your Key Objectives:
> - Create specialized converters using closures (functions that remember settings).
> - Track events for each probe with its own private log (also closures).
> - You write and read simple mission logs from text files.
> - Use `bytearray` to tweak binary headers before sending them.

## Part 1: Distance Converter Factory (closure)

Context: To kick off this mission, we'll build a function that creates custom distance converters.

Your Tasks:
- [ ] 1.1 - Implement a function `create_distance_converter` that takes in input parameters `target_unit` and `factor`.
- [ ] 1.2 - Inside it, define an inner function `convert(distance)` that:
  - multiplies `distance` by `factor`
  - returns the result
- [ ] 1.3 - Return the inner `convert` function.  
  - Because `convert` uses `factor` from the outer scope, it should behave as a closure that “remembers” the factor even after the factory returns.

In [ ]:
# === Part 1: Your code here ===

# 1.1)
def create_distance_converter(target_unit, factor):
    """
    Create a converter that multiplies distances by a fixed factor.
    The returned function is a closure that remembers 'factor'.
    """
    # 1.2)
    def convert(distance):
        return distance * factor
    # 1.3)
    return convert

## Part 2: Probe Event Logger (closure with Private State)

Context: Each probe should keep its own list of events. We'll use a closure so each logger has its own private list.

Your Tasks:
- [ ] 2.1 - Implement a new `make_probe_logger` function that passes in input parameter `probe_name`.
- [ ] 2.2 - Inside it, create an empty list called `events`
- [ ] 2.3 - Define an inner function `log_event(message)` that:
  - appends `message` to `events`
  - returns a string like `"Voyager 1: ['LAUNCHED', 'IN CRUISE']"`
- [ ] 2.4 - Return `log_event`.  
  - Because `log_event` closes over `events`, each call to `make_probe_logger(...)` should get its own independent `events` list.

In [ ]:
# === Part 2: Your code here ===

# 2.1)
def make_probe_logger(probe_name):
    """
    Create a logger function that remembers its probe name and a private list of events.
    """
    # 2.2)
    events = []

    # 2.3)
    def log_event(message):
        events.append(message)
        return f"{probe_name}: {events}"

    # 2.4)
    return log_event

## Part 3: Simple Mission Log File (Text I/O)

Context: Next, we'll write a few lines to a mission log file, and then read them back to confirm a successful transmission.

Your Tasks:
- [ ] 3.1 - Implement `write_mission_log(filename, lines)`:
  - Use a `with open(..., 'w', encoding='utf-8') as f:` block.
  - For each string in `lines`, write it with a trailing `"\n"`.
- [ ] 3.2 - Implement `read_mission_log(filename)`:
  - Use `with open(..., 'r', encoding='utf-8') as f:`.
  - Use `readlines()` to read all lines.
  - Strip the trailing newline from each line (use `.strip()` in a list comprehension).
  - Return the resulting list of strings.

In [ ]:
# === Part 3: Your code here ===

# 3.1)
def write_mission_log(filename, lines):
    """
    Write each string from 'lines' to the file, one per line.
    """
    with open(filename, "w", encoding="utf-8") as f:
        for line in lines:
            f.write(line + "\n")


# 3.2)
def read_mission_log(filename):
    """
    Read all lines from the file and return them without trailing newlines.
    """
    with open(filename, "r", encoding="utf-8") as f:
        raw_lines = f.readlines()
    cleaned = [line.strip() for line in raw_lines]
    return cleaned

## Part 4: Prepare a binary header with `bytearray`

Context: Now, you're about to send a binary message. We want to build a mutable header that:
- Starts from a text label (ASCII only).
- Ends with a newline byte (`\n`, value 10).

Your Tasks:
- [ ] 4.1 - Implement a `make_header` function that passes in input parameter `label`.
- [ ] 4.2 - Inside this function, convert `label` to bytes using ASCII (e.g. `label.encode("ascii")`), and wrap that in `bytearray(...)` to make it mutable.
- [ ] 4.3 - If the header does not already end with `b"\n"` (byte value `10`), append the newline byte.
- [ ] 4.4 - Return the resulting `bytearray`.

In [ ]:
# === Part 4: Your code here ===

# 4.1)
def make_header(label):
    """
    Create a mutable header from an ASCII label, ensuring it ends with a newline byte.
    """
    # 4.2)
    header = bytearray(label.encode("ascii"))
    # 4.3)
    if not header or header[-1] != 10:  # 10 == ord('\n')
        header.append(10)
    # 4.4)
    return header

## Part 5: Mission Run — Exercise All Paths

Context: Once you've implemented Parts 1-3, run the cells below without modification. They'll help you confirm that your helpers behave correctly for different kinds of text.


In [ ]:
# === Tests for Part 1: distance converter factory ===

# 1 AU ≈ 149.6 million km (we'll invert that)
km_to_au = create_distance_converter("AU", 1 / 149.6)
km_to_lightyears = create_distance_converter("ly", 1 / 9.461e12)

print("Earth distance (km -> AU):", km_to_au(149.6))      # Expected: Earth distance (km -> AU): 0.9999999999999999
print("10^9 km in light-years:", km_to_lightyears(1e9))   # Expected: 10^9 km in light-years: 0.00010569707219110031

Earth distance (km -> AU): 0.9999999999999999
10^9 km in light-years: 0.00010569707219110031


In [ ]:
# === Tests for Part 2: probe event logger (closures with state) ===

voyager = make_probe_logger("Voyager 1")
apollo = make_probe_logger("Apollo 11")

print(voyager("LAUNCHED"))        # Expected: Voyager 1: ['LAUNCHED']
print(voyager("IN CRUISE"))       # Expected: Voyager 1: ['LAUNCHED', 'IN CRUISE']
print(apollo("LAUNCHED"))         # Expected: Apollo 11: ['LAUNCHED']
print(voyager("NEAR JUPITER"))    # Expected: Voyager 1: ['LAUNCHED', 'IN CRUISE', 'NEAR JUPITER']
print(apollo("LUNAR ORBIT"))      # Expected: Apollo 11: ['LAUNCHED', 'LUNAR ORBIT']

Voyager 1: ['LAUNCHED']
Voyager 1: ['LAUNCHED', 'IN CRUISE']
Apollo 11: ['LAUNCHED']
Voyager 1: ['LAUNCHED', 'IN CRUISE', 'NEAR JUPITER']
Apollo 11: ['LAUNCHED', 'LUNAR ORBIT']


In [ ]:
# === Tests for Part 3: mission log file (text I/O) ===

log_filename = "mission_log_l6.txt"

entries = [
    "Mission start",
    "Stage separation",
    "Orbit achieved",
]

write_mission_log(log_filename, entries)
read_back = read_mission_log(log_filename)

print("Wrote:", entries)                      # Expected: Wrote: ['Mission start', 'Stage separation', 'Orbit achieved']
print("Read back:", read_back)                # Expected: Read back: ['Mission start', 'Stage separation', 'Orbit achieved']

Wrote: ['Mission start', 'Stage separation', 'Orbit achieved']
Read back: ['Mission start', 'Stage separation', 'Orbit achieved']


In [ ]:
# === Tests for Part 4: binary header with bytearray ===

h1 = make_header("TELEMETRY")
h2 = make_header("EVENTS\n")

print("Header 1:", h1, "last byte:", h1[-1])    # Expected: Header 1: bytearray(b'TELEMETRY\n') last byte: 10
print("Header 2:", h2, "last byte:", h2[-1])    # Expected: Header 2: bytearray(b'EVENTS\n') last byte: 10
# Both should end with newline byte (10)

Header 1: bytearray(b'TELEMETRY\n') last byte: 10
Header 2: bytearray(b'EVENTS\n') last byte: 10


## Part 6: Debrief (short answers)

### Q1 of 2: How does a closure like `make_probe_logger(...)` remember its `probe_name` and `events` list?

The inner function `log_event` is defined inside `make_probe_logger`, so it "closes over" the variables from that outer scope (`probe_name` and `events`).

Even after `make_probe_logger` finishes running, the returned `log_event` function still has a reference to those variables stored in its closure. So, it can keep appending to the same `events` list and keep using the same `probe_name` every time you call it, which highlights the immense value that closures have as we write our Python scripts.

_______

### Q2 of 2: Why might you choose `bytearray` over an immutable `bytes` object when preparing binary data for I/O?

You might do this because `bytearray`'s are mutable, while `bytes` are not.

This means that with a `bytearray`, you can:
- Modify bytes in place (e.g., tweak headers, flags, checksums),
- Reuse the same buffer for repeated reads/writes,
- Avoid creating lots of new objects when changing small parts of the data.

This makes `bytearray`'s more memory-efficient and better for performance in I/O-heavy situations or streaming scenarios where you're constantly updating binary data.

_______